- FLOP（每秒浮点运算）通过计算执行的浮点运算数量来测量神经网络模型的计算复杂性
- 高FLOP表示更密集的计算和能耗

In [2]:
from importlib.metadata import version

import matplotlib
import torch
import torch.nn as nn

print(f"thop version:{version('thop')}")
print(f"torch version:{version('torch')}")

thop version:0.1.1-2209072238
torch version:2.3.1


In [4]:
import torch
from thop import profile

from GPT import GPTModel

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt-small (124M)": {"embedding_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt-medium (355M)": {"embedding_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt-large (774M)": {"embedding_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt-xl (1558M)": {"embedding_dim": 1600, "n_layers": 48, "n_heads": 25},
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = torch.randint(0, 50257, (2, 1024)).to(device)

for size in model_configs:
    BASE_CONFIG.update(model_configs[size])
    model = GPTModel(BASE_CONFIG).bfloat16()
    model.to(device)
    
    macs, params = profile(model, inputs=(input_tensor, ), verbose=False)
    flops = 2 * macs
    print(f"{size:18}: {flops:.le} FLOPS")
    
    del model
    torch.cuda.empty_cache()

ValueError: Format specifier missing precision